In [46]:
import os
import sys

from arcgis import env
from arcgis.gis import GIS
import arcpy
from dotenv import load_dotenv, find_dotenv
import requests

sys.path.append('../src')
import dm

# load environment variables from .env
load_dotenv(find_dotenv())

import os.path
from pathlib import Path
import sys

from arcgis.gis import GIS
import arcpy
import pandas as pd

sys.path.insert(0, os.path.abspath('../'))
from dm import Country
from dm._registry import get_ba_key_value

In [47]:
# DMA to test against
test_dma_name = 'seattle'

# get paths to data for stress testing
ba_data_dir = Path(get_ba_key_value('DataInstallDir', 'USA'))
gdb = ba_data_dir/'Data'/'Demographic Data'/'USA_ACS_2019.gdb'
bg_fc = gdb/'BlockGroups_bg'
dma_fc = gdb/'DMAs_dm'

# create a couple of layers for working with
dma_lyr = arcpy.management.MakeFeatureLayer(str(dma_fc), where_clause=f"UPPER(NAME) LIKE UPPER('%{test_dma_name}%')")
assert int(arcpy.management.GetCount(dma_lyr)[0]) == 1
bg_lyr = arcpy.management.MakeFeatureLayer(str(bg_fc))

# select only the block groups in the DMA
arcpy.management.SelectLayerByLocation(
    in_layer=bg_lyr,
    overlap_type='HAVE_THEIR_CENTER_IN',
    select_features=dma_lyr
)     

id,value
0,a Layer object
1,BlockGroups_bg_Layer
2,3390


In [48]:
usa = dm.Country('USA')

In [49]:
n_vars = 20

enrich_vars = usa.enrich_variables.enrich_str

var_lst = enrich_vars.sample(n_vars)
 
var_str = ';'.join(var_lst)

var_str

'tapestryhouseholdsNEW.THH32;transportation.X6062FY_X;agebyracebysex.AIF45_CY;clothing.X5059_X;agebyracebysex.MLTF35_FY;raceandhispanicorigin.RACE2UP_CY;agebyracebysex.ASNM30_FY;HealthPersonalCare.MP14016a_B;agebyracebysex.ASN35C10;HousingHousehold.X4099_X;agebyracebysex.HSP45_FY;GroceryAlcoholicBeverages.MP12001h_B;1yearincrements.AGE72_FY;PsychographicsAdvertising.MP28166a_B;PetsPetProducts.MP26012h_B;agebyracebysex.ASNF75_FY;vehiclesavailable.ACSOVEH3;1yearincrements.MAGE54_CY;GroceryAlcoholicBeverages.MP13025a_B;MediaRadioOtherAudio.MP22086a_B'

In [50]:
res = arcpy.ba.EnrichLayer(bg_lyr,
                           os.path.join(arcpy.env.scratchGDB, f'test_enrich_{len(var_lst):04d}'),
                           var_str)[0]

res

'C:\\Users\\joel5174\\AppData\\Local\\Temp\\scratch.gdb\\test_enrich_0020'